<a href="https://colab.research.google.com/github/sanghoho/Poverty-Satellite/blob/master/satellite_image_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 드라이브 연동

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive/', force_remount=True)
os.chdir("gdrive/My Drive/20-satellite-olympic")

## EasyEarh Class 정의

구름별 image sort 기준
1. Copernicus : "CLOUDY_PIXEL_PERCENTAGE"
2. Landsat : "CLOUD_COVER"


In [ ]:
import ee
import math

import requests
from PIL import Image
from io import BytesIO

try:
  ee.initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize()


class EasyEarth:
  def __init__(self, dataset):
    self.dataset = ee.ImageCollection(dataset)

  def change_dataset(self, dataset):
    self.dataset = ee.ImageCollection(dataset)

  def select_AOI(self, lat, lon, k = 10, dates=None, cloud_name = "CLOUD_COVER" ,cloud_pct=None):
    self.outer_AOI = self.__create_AOI(lat, lon, k / 50)
    self.AOI = self.__create_AOI(lat, lon, k)
    self.dates = dates
    #해당 지역이 포함된 위성 사진 가져오기.
    self.data_AOI = self.dataset.filterBounds(self.AOI)
    #날짜 필터
    if dates is not None:
      self.data_AOI = self.__filter_dates(self.data_AOI, dates)
    #구름필터
    if cloud_pct is not None:
      self.data_AOI = self.__filter_cloudy(by=cloud_name, cloud_pct=cloud_pct)
    self.AOI_size = self.data_AOI.size().getInfo()
    #self.data_AOI = self.data_AOI.map(cloudMask)

  #대기효과 보정(구름제거)
  def cloudMask(image) :
    qa = image.select('QA60')
    allCloudBitMask = (1 << 10) + (1 << 11)
    mask = qa.bitwiseAnd(allCloudBitMask).eq(0)
    return image.updateMask(mask);

  def __create_AOI(self, lat, lon, s=10):
    """Creates a s km x s km square centered on (lat, lon)"""
    v = (180/math.pi)*(500/6378137)*s # roughly 0.045 for s=10
    self.geometry = ee.Geometry.Polygon([[[lon - v, lat + v],
                                     [lon - v, lat - v],
                                     [lon + v, lat - v],
                                     [lon + v, lat + v]]], None, False)
    return self.geometry #ee.Geometry.Rectangle([lon - v, lat - v, lon + v, lat + v])
  def create_geo(self,lat,lon,s):
    v = (180/math.pi)*(500/6378137)*s # roughly 0.045 for s=10
    self.geometry = ee.Geometry.Polygon([[[lon - v, lat + v],
                                     [lon - v, lat - v],
                                     [lon + v, lat - v],
                                     [lon + v, lat + v]]], None, False)
    return self.geometry
  def __filter_dates(self, col, dates):
    return col.filterDate(dates[0], dates[1])


  def __filter_cloudy(self, by="CLOUDY_PIXEL_PERCENTAGE", cloud_pct=5):
    """
    주로 Sentinel(COPERNIQUS)에서 구름 양을 조절하기 위해 사용합니다.
    """
    return self.data_AOI.filter(ee.Filter.lt(by, cloud_pct))
            

  def sort_by(self, method = 'CLOUD_COVER', ascending=True):
    sorted_collections = self.data_AOI.sort(method, ascending)
    self.data_AOI = sorted_collections


  def plot_image(self, img, paramters, cloud_name = 'CLOUD_COVER'):
    
    cloud_pct = img.get(cloud_name).getInfo()
    # print(f'Cloud Cover (%): {cloud_pct}')

    url = img.getThumbUrl(parameters)
    response = requests.get(url)
    
    return cloud_pct, Image.open(BytesIO(response.content))


  def get_collections_at(self, idx, collections=None):
    if collections is not None:
      size = collections.size().getInfo()
      return ee.Image(collections.toList(size).get(idx)) 
    else:
      return ee.Image(self.data_AOI.toList(self.AOI_size).get(idx))
            
  def save_image(self, save_path, image):
    plt.imsave(save_path, np.array(image))
    # print(str(save_path) + " complete")

  def calculate_alpha_ratio(self, image):
    img_array = np.array(image)
    return np.sum(img_array == 0) / img_array.size
    
  def get_ndvi(self,image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    region = self.geometry
    # Use a mean reducer.
    reducer = ee.Reducer.mean()

    # Compute the unweighted mean.
    ndvi_value = ndvi.select(['NDVI']).reduceRegion(reducer, region, 10)
    return ndvi_value.getInfo()['NDVI']
  def get_ndbi(self,image):
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
    region = self.geometry
    # Use a mean reducer.
    reducer = ee.Reducer.mean()

    # Compute the unweighted mean.
    ndbi_value = ndbi.select(['NDBI']).reduceRegion(reducer, region, 10)
    return ndbi_value.getInfo()['NDBI']
  
  def get_ui(self,image):
     Ndvi = image.normalizedDifference(['B8', 'B4'])
     Ndbi = image.normalizedDifference(['B11', 'B8'])
     Ui = Ndbi.subtract(Ndvi).rename('Ui');
     region = self.geometry
     reducer = ee.Reducer.mean()
     Ui_value = Ui.select(['Ui']).reduceRegion(reducer, region, 10)
     return Ui_value.getInfo()['Ui']
  

In [ ]:
def create_geo(lat,lon,s = 10):
    v = (180/math.pi)*(500/6378137)*s # roughly 0.045 for s=10
    geometry = ee.Geometry.Polygon([[[lon - v, lat + v],
                                    [lon - v, lat - v],
                                    [lon + v, lat - v],
                                    [lon + v, lat + v]]], None, False)
    return geometry


#평창 군청
lat = 37.370814 
lon = 128.390359

a = create_geo(lat,lon)
a

In [ ]:
v = (180/math.pi)*(500/6378137)*10 # roughly 0.045 for s=10
a = ee.Geometry.Rectangle([lon - v, lat - v, lon + v, lat + v])
print(a)

In [ ]:
parameters = {
  'min': 0.0,
  'max': 2500,
  'bands': ['B4', 'B3', 'B2'],
  'dimensions': 512,
  'region': earth.AOI
}

earth.sort_by("CLOUDY_PIXEL_PERCENTAGE")

least_cloudy_img = earth.get_collections_at(1)

cloud , img = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, "CLOUDY_PIXEL_PERCENTAGE")

print(cloud)
img

In [ ]:
least_cloudy_img.toArray()

In [ ]:
nir = least_cloudy_img.select('B8')
red = least_cloudy_img.select('B4')
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

### Sentinel 위성 이미지 추출

In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
earth = EasyEarth('COPERNICUS/S2')
CLOUD_NAME = "CLOUDY_PIXEL_PERCENTAGE" # "CLOUD_COVER"
#알펜시아
#lat = 37.658251
#lon = 128.669856
#진부역
#lat = 37.642935
#lon = 128.574451
#올림픽 게이트웨이
#lat = 37.668117
#lon = 128.705676
#자연휴양
#lat = 37.705692
#lon = 128.720683
#평창역
#lat = 37.562846
#lon = 128.429930
#평창군청
lat = 37.370814 
lon = 128.390359
save_df = pd.DataFrame(columns=["raw_name", "lat", "lon", "image_name", "cloud_pct","year","time","NDVI","NDBI","UI"])
image_list = []
for year in ["2016","2017","2018","2019","2020"]:
  name = f"{lat}_{lon}_{year}.png" 
  if(year == "2017"):
    earth.select_AOI(lat,lon, 10, ("2017-04-15", "2017-05-01"), CLOUD_NAME, 10)
  elif (year =="2018"):
    earth.select_AOI(lat,lon, 10, ("2018-04-01", "2018-04-19"), CLOUD_NAME, 10)
  elif (year == '2020'):
    earth.select_AOI(lat,lon, 10, ("2020-04-28", "2020-04-30"), CLOUD_NAME, 10)
  else :
    earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)
  
  earth.data_AOI.map(cloudMask)
  parameters = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    'dimensions': 512,
    'region': earth.AOI
  }
  earth.sort_by(CLOUD_NAME)
  least_cloudy_img = earth.get_collections_at(0)
  ndvi = earth.get_ndvi(least_cloudy_img)
  ndbi = earth.get_ndbi(least_cloudy_img)
  ui = earth.get_ui(least_cloudy_img)
  cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
  image_list.append(least_result)
  date = ee.Date(least_cloudy_img.get('system:time_start'))
  time = date.getInfo()['value']/1000
  
  # 메타 정보 데이터프레임에 저장
  tmp_result = pd.Series({"raw_name" : least_cloudy_img.getInfo()['id'],"lat": lat, "lon": lon,  "image_name": name,  "cloud_pct": cloud_pct,
                            "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'),"NDVI" : ndvi,"NDBI" : ndbi,"UI" : ui}).to_frame().T
  save_df = save_df.append(tmp_result,ignore_index=True)

In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
earth = EasyEarth('COPERNICUS/S2')
CLOUD_NAME = "CLOUDY_PIXEL_PERCENTAGE" # "CLOUD_COVER"
#알펜시아
#lat = 37.658251
#lon = 128.669856
#진부역
#lat = 37.642935
#lon = 128.574451
#올림픽 게이트웨이
#lat = 37.668117
#lon = 128.705676
#자연휴양
#lat = 37.705692
#lon = 128.720683
#평창역
#lat = 37.562846
#lon = 128.429930
#평창군청
lat = 37.370814 
lon = 128.390359
save_df = pd.DataFrame(columns=["raw_name", "lat", "lon", "image_name", "cloud_pct","year","time","NDVI","NDBI","UI"])
image_list = []
earth.select_AOI(lat,lon, 10, range_year(2016), CLOUD_NAME, 10)
parameters = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    'dimensions': 512,
    'region': earth.AOI
  }
earth.sort_by(CLOUD_NAME)
least_cloudy_img = earth.get_collections_at(0)
ndvi = earth.get_ndvi(least_cloudy_img)
ndbi = earth.get_ndbi(least_cloudy_img)
ui = earth.get_ui(least_cloudy_img)
cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
image_list.append(least_result)
date = ee.Date(least_cloudy_img.get('system:time_start'))
time = date.getInfo()['value']/1000



In [ ]:


task = ee.batch.Export.image.toDrive(**{
  'image': least_cloudy_img,
  'description': 'imageToCOGeoTiffExample',
  'scale': 30,
  'fileFormat': 'GeoTIFF',
  'formatOptions': {
    'cloudOptimized': True 
  }
});

task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image = least_cloudy_img,description = 'didididi')

task.start()

In [ ]:
landsat = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_123032_20140515').select(['B4', 'B3', 'B2']);

geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236]);

ee.batch.Export.image.toDrive(**{
  'image': landsat,
  'description': 'imageToDriveExample',
  'scale': 30,
});

google earth에서 행정구역 대로 자르기(google earth engine code editor에 들어가서 asset에 shapefile을 업로드 해야한다)

참고 자료: https://developers.google.com/earth-engine/importing

In [ ]:
table= ee.FeatureCollection("users/jinwoo95/pyeongchang")
area = table.getInfo()['features'][0]['geometry']
collection = ee.ImageCollection("COPERNICUS/S2").filterBounds(area)\
                                      .filterDate("2018-01-01","2019-01-10")\
                                      .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",10)\
                                      .select(['B8', 'B4'])
print(" number of image: ",collection.size().getInfo())

In [ ]:
table.map

Test

In [ ]:
from osgeo import gdal
from matplotlib import pyplot as plt
import sys
import numpy as np

In [ ]:
earth = EasyEarth('COPERNICUS/S2')
lat = 37.5624
lon = 128.4293
##위도,경도,반경 몇키로, 날짜, image_sort기준, 구름 몇 %이하로 설정할 것인지
earth.select_AOI(lat,lon, 5, ("2018-04-01", "2018-10-31"), "CLOUDY_PIXEL_PERCENTAGE", 10)
parameters = {
  'min': 0.0,
  'max': 2500,
  'bands': ['B4', 'B3', 'B2'],
  'dimensions': 512,
  'region': earth.AOI
}

earth.sort_by("CLOUDY_PIXEL_PERCENTAGE")

least_cloudy_img = earth.get_collections_at(0)

cloud , img = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, "CLOUDY_PIXEL_PERCENTAGE")


In [ ]:
earth = EasyEarth('COPERNICUS/S2')
lat = 37.5624
lon = 128.4293
geometry = earth.create_geo(lat,lon,10)
'''year = 2019
image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(geometry)
              .filterDate('2017-04-15','2017-05-15')
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .first()
              .clip(geometry))
              '''

# 새 섹션

In [ ]:
earth = EasyEarth('COPERNICUS/S2')
image = (earth.dataset
                 .filterBounds(geometry)
                 .filterDate('2020-04-15','2020-05-15')
                .sort('CLOUDY_PIXEL_PERCENTAGE')
                .first()
                .clip(geometry))
                 
                  

In [ ]:
image.getInfo()['id']

In [ ]:
least_cloudy_img.getInfo()

위성 영상 전처리 in google earth : https://developers.google.com/earth-engine/landsat

위성 영상 전처리 개념 정리 글 : https://m.blog.naver.com/PostView.nhn?blogId=rsmilee&logNo=220648092210&proxyReferer=https:%2F%2Fwww.google.com%2F

머신러닝을 이용한 위성사진 분류 (supervised learning)  

구글 어스 엔진을 사용한 논문 : https://www.frontiersin.org/articles/10.3389/feart.2017.00017/full

1) Naive Bayes

---
process sequence

1. google earth engine에서 그리기를 통해 직접 훈련 데이터 셋을 만든다(직접 만드는 방법 말고 훈련 데이터 집합 이미지를 직접 저장해놓고 있는 경우도 있다). 각각의 클래스에 해당하는 픽셀을 여러개 지정한다.  

2. classifier를 설정한다. 필요하다면 파라미터도 설정하지만 Naive-Bayes 에서는 지정해야할 파라미터가 없기 때문에 생략한다.

3. 훈련 데이터셋을 통해서 classifier를 학습시킨다.

4. 이미지 데이터 셋 분류를 실시한다.

5. validation set을 통해 분류 오류율을 체크한다.



In [ ]:
#시각화 패키지 설치
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

In [ ]:
#동적 맵 생성하기.
Map = geemap.Map(center=[40,-100], zoom=4)
Map

# 분류하기

1. 인풋이 되는 위성 사진(TIF)을 추출한다.
2. 딥러닝을 위해 해당 JPG사진을 다운로드 한다.

In [ ]:
#분류 전 데이터 학습 및 검증하기
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
def range_year(year):
  return f"{year}-04-15", f"{year}-05-15"
earth = EasyEarth('COPERNICUS/S2')
CLOUD_NAME = "CLOUDY_PIXEL_PERCENTAGE" # "CLOUD_COVER"
#알펜시아
#lat = 37.658251
#lon = 128.669856
#진부역
#lat = 37.642935
#lon = 128.574451
#올림픽 게이트웨이
#lat = 37.668117
#lon = 128.705676
#자연휴양
#lat = 37.705692
#lon = 128.720683
#평창역
lat = 37.562846
lon = 128.429930

save_df = pd.DataFrame(columns=["lat", "lon", "image_name",  "cloud_pct","year","time","NDVI","NDBI"])
image_list = []

for year in ["2016","2017","2018","2019","2020"]:
  
  name = f"{lat}_{lon}_{year}.png" 
  if(year == "2017"):
    earth.select_AOI(lat,lon, 10, ("2017-04-12", "2017-04-15"), CLOUD_NAME, 10)
  else :
    earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)
  
  parameters = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    'dimensions': 512,
    'region': earth.AOI
  }
  earth.sort_by(CLOUD_NAME)

  least_cloudy_img = earth.get_collections_at(0)
  ndvi = earth.get_ndvi(least_cloudy_img)
  ndbi = earth.get_ndbi(least_cloudy_img)
  ui = earth.get_ui(least_cloudy_img)
  cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
  image_list.append(least_result)
  date = ee.Date(least_cloudy_img.get('system:time_start'))
  time = date.getInfo()['value']/1000.
  
  # 메타 정보 데이터프레임에 저장
  tmp_result = pd.Series({"lat": lat, "lon": lon,  "image_name": name,  "cloud_pct": cloud_pct,
                            "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'),"NDVI" : ndvi,"NDBI" : ndbi,"UI" : ui}).to_frame().T
  save_df = save_df.append(tmp_result,ignore_index=True)

In [ ]:
image_list[4]

In [ ]:
#위성 이미지 불러오기
earth = EasyEarth('COPERNICUS/S2')
lat = 37.647125
lon = 128.685141
##위도,경도,반경 몇키로, 날짜, image_sort기준, 구름 몇 %이하로 설정할 것인지
earth.select_AOI(lat,lon, 100, ("2020-04-01", "2020-04-30"), "CLOUDY_PIXEL_PERCENTAGE", 10)
parameters = {
  'min': 0.0,
  'max': 2500,
  'bands': ['B4', 'B3', 'B2'],
  'dimensions': 512,
  'region': earth.AOI
}
earth.sort_by("CLOUDY_PIXEL_PERCENTAGE")

image = earth.get_collections_at(0)
##training set 가져오기
points= ee.FeatureCollection("users/jinwoo95/training_point")

#area = table.getInfo()['features'][0]
bands = ["B2","B3","B4","B5","B6","B7"]
#label은 훈련 데이터 셋에 각 class를 지정할 때 property명임.
label = 'lc'
#train, test_set 나누기.

data_set = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 1000
})

sample = data_set.randomColumn()
split = 0.7  
training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

# Train a CART classifier with default parameters.
classifier = ee.Classifier.cart().train(**{
 'features': training , 
  'classProperty': label,
  'inputProperties': bands
})
# Classify the image with the same bands used for training.
classified = image.select(bands).classify(classifier)

# Display the inputs and the results.
#Map.centerObject(points, 11)
#Map.addLayer(classified,
#             {'min': 0, 'max': 3, 'palette': ['#0b4a8b', '#ffc82d', '#00ffff','#bf04c2','#ff0000']},
#           'classification')

#Map
trainAccuracy = classifier.confusionMatrix();

validated = validation.classify(classifier)
testAccuracy = validated.errorMatrix('lc', 'classification')

In [ ]:
#위성 이미지 불러오기
earth = EasyEarth('COPERNICUS/S2')
lat = 37.647125
lon = 128.685141
##위도,경도,반경 몇키로, 날짜, image_sort기준, 구름 몇 %이하로 설정할 것인지
ee.ImageCollection('COPERNICUS/S2_SR')
                .filterBounds(region)
                .filterDate('2019-04-01', '2019-04-30')
                .sort('CLOUDY_PIXEL_PERCENTAGE')
                .first();
earth.select_AOI(lat,lon, 10, ("2020-04-01", "2020-04-30"), "CLOUDY_PIXEL_PERCENTAGE", 10)
parameters = {
  'min': 0.0,
  'max': 2500,
  'bands': ['B4', 'B3', 'B2'],
  'dimensions': 512,
  'region': earth.AOI
}
earth.sort_by("CLOUDY_PIXEL_PERCENTAGE")

image = earth.get_collections_at(0)

In [ ]:
cloud_pct, least_result = earth.plot_image(image.resample('bicubic'), parameters, CLOUD_NAME)

In [ ]:
table= ee.FeatureCollection("users/jinwoo95/pyeongchang")
#table.getInfo()
#area = table.getInfo()['features'][0]['geometry']['coordinates']
type(table.getInfo())

### Decision Tree 학습법

In [ ]:
region = ee.Geometry.Polygon([[[
                128.30838423579402,
                37.480315764205976
              ],
              [
                128.30838423579402,
                37.39048423579403
              ],
              [
                128.39821576420596,
                37.39048423579403
              ],
              [
                128.39821576420596,
                37.480315764205976
              ]
            ]])
#평창군 행정구역
table= ee.FeatureCollection("users/jinwoo95/pyeongchang")
area = table.getInfo()['features'][0]['geometry']['coordinates']
roi = ee.Geometry.Polygon(area)
image = (ee.ImageCollection('COPERNICUS/S2')
                .filterBounds(region)
                .filterDate('2019-04-01', '2019-04-30')
                .sort('CLOUDY_PIXEL_PERCENTAGE')
                .first()
                .clip(roi))
##data set 가져오기
points= ee.FeatureCollection("users/jinwoo95/revised_training3")

#area = table.getInfo()['features'][0]
#bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B9","B11","B12"]
#bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B10","B11","B12"]
bands = ["B4","B8","B11"]
#label은 훈련 데이터 셋에 각 class를 지정할 때 property명임.
label = 'class'
#train, test_set 나누기.

data_set = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 10
})

sample = data_set.randomColumn()
split = 0.7  
training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

# Train a CART classifier with default parameters.
classifier = ee.Classifier.cart().train(**{
 'features': training , 
  'classProperty': label,
  'inputProperties': bands
})
# Classify the image with the same bands used for training.
classified = image.select(bands).classify(classifier)

# Display the inputs and the results.
#Map.centerObject(points, 11)
#Map.addLayer(classified,
#             {'min': 0, 'max': 3, 'palette': ['#0b4a8b', '#ffc82d', '#00ffff','#bf04c2','#ff0000']},
#           'classification')

#Map
trainAccuracy = classifier.confusionMatrix();

validated = validation.classify(classifier)
testAccuracy = validated.errorMatrix('class', 'classification')
#image = image2.select('B4', 'B3', 'B2')
#Map.addLayer(image, {'gain': [1.4, 1.4, 1.1]}, 'Landsat 7')

In [ ]:
print(data_set.size().getInfo())
#print(training.size().getInfo())
#print(validation.size().getInfo())

In [ ]:
print(trainAccuracy.accuracy().getInfo())
#print(trainAccuracy.getInfo())

In [ ]:
print(testAccuracy.accuracy().getInfo())

In [ ]:
print(testAccuracy.getInfo())

### Random Forest

In [ ]:
# Train a CART classifier with default parameters.
classifier = (ee.Classifier.smileRandomForest(10)
    .train(**{
      'features': training,
      'classProperty': label,
      'inputProperties': bands}));
# Classify the image with the same bands used for training.
classified = image.select(bands).classify(classifier)

# Display the inputs and the results.
#Map.centerObject(points, 11)
#Map.addLayer(classified,
#             {'min': 0, 'max'  : 3, 'palette': ['#0b4a8b', '#ffc82d', '#00ffff','#bf04c2','#ff0000']},
#           'classification')

#Map
trainAccuracy = classifier.confusionMatrix();

validated = validation.classify(classifier)
testAccuracy = validated.errorMatrix('class', 'classification')
#image = image2.select('B4', 'B3', 'B2')
#Map.addLayer(image, {'gain': [1.4, 1.4, 1.1]}, 'Landsat 7')

In [ ]:
print(trainAccuracy.accuracy().getInfo())
#print(trainAccuracy.getInfo())

In [ ]:
print(testAccuracy.accuracy().getInfo())

In [ ]:
print(testAccuracy.getInfo())

### SVM

In [ ]:

"""classifier = (ee.Classifier.libsvm(**{
  'kernelType': 'RBF',
  'gamma': 0.5,
  'cost': 10})
    .train(**{
      'features': training,
      'classProperty': label,
      'inputProperties': bands}));"""
classifier = (ee.Classifier.libsvm()
    .train(**{
      'features': training,
      'classProperty': label,
      'inputProperties': bands}));
# Classify the image with the same bands used for training.
classified = image.select(bands).classify(classifier)
trainAccuracy = classifier.confusionMatrix();

validated = validation.classify(classifier)
testAccuracy = validated.errorMatrix('class', 'classification')

In [ ]:
print(trainAccuracy.accuracy().getInfo())
#print(trainAccuracy.getInfo())

In [ ]:
print(testAccuracy.accuracy().getInfo())

In [ ]:
print(testAccuracy.getInfo())

### Naive Bayes

In [ ]:
classifier = (ee.Classifier.naiveBayes().train(**{
      'features': training,
      'classProperty': label,
      'inputProperties': bands}));
# Classify the image with the same bands used for training.
classified = image.select(bands).classify(classifier)
trainAccuracy = classifier.confusionMatrix();

validated = validation.classify(classifier)
testAccuracy = validated.errorMatrix('class', 'classification')

In [ ]:
print(trainAccuracy.accuracy().getInfo())
#print(trainAccuracy.getInfo())

In [ ]:
print(testAccuracy.accuracy().getInfo())

In [ ]:
print(testAccuracy.getInfo())

## 특정 구역 연도별 class 변화 살펴보기(random forest 사용)

### 1.알펜시아 용평 리조트

In [ ]:
#분류 전 데이터 학습 및 검증하기
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
def range_year(year):
  return f"{year}-04-15", f"{year}-05-15"
#알펜시아
lat = 37.658251
lon = 128.669856
#진부역
#lat = 37.642935
#lon = 128.574451
#올림픽 게이트웨이
#lat = 37.668117
#lon = 128.705676
#자연휴양
#lat = 37.705692
#lon = 128.720683
#평창역
#lat = 37.562846
#lon = 128.429930
earth = EasyEarth('COPERNICUS/S2')
CLOUD_NAME = "CLOUDY_PIXEL_PERCENTAGE" # "CLOUD_COVER"

bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']
save_df = pd.DataFrame(columns=["lat", "lon", "image_name","cloud_pct","year","time","Classified","Geometry"])
image_list = []

for year in ["2016","2017","2018","2019","2020"]:
  
  name = f"{lat}_{lon}_{year}.png" 
  if(year == "2017"):
    earth.select_AOI(lat,lon, 10, ("2017-04-12", "2017-04-15"), CLOUD_NAME, 10)
  else :
    earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)
  
  parameters = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    'dimensions': 512,
    'region': earth.AOI
  }
  
  geometry = earth.AOI
  earth.sort_by(CLOUD_NAME)
  least_cloudy_img = earth.get_collections_at(0)
  least_cloudy_img = least_cloudy_img.clip(earth.AOI)
  classified = least_cloudy_img.select(bands).classify(classifier)
  cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
  image_list.append(least_result)
  date = ee.Date(least_cloudy_img.get('system:time_start'))
  time = date.getInfo()['value']/1000.
  
  #학습된 모델로 이미지 분류

  # 메타 정보 데이터프레임에 저장
  tmp_result = pd.Series({"lat": lat, "lon": lon,  "image_name": name,  "cloud_pct": cloud_pct,
                            "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'),"Classified" :classified,"Geometry":geometry}).to_frame().T
  save_df = save_df.append(tmp_result,ignore_index=True)


In [ ]:
#save_df['Classified'][0].getInfo()
#classified.getInfo()
least_result

In [ ]:
save_df['Classified'][4].getInfo()

In [ ]:
#image = ee.Image(collection.sort('CLOUD_COVER').first()
lat = 37.658251
lon = 128.669856
CLOUD_NAME='CLOUDY_PIXEL_PERCENTAGE'
earth = EasyEarth('COPERNICUS/S2_SR')
geometry = earth.create_geo(lat,lon,10)
year = 2019
image = (ee.ImageCollection('COPERNICUS/S2_SR')
              .filterBounds(geometry)
              #.filterDate(f'{year}-04-14', f'{year}-05-15')
              .filterDate('2017-04-13','2017-04-14')
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .first()
              .clip(geometry))
parameters = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
  'dimensions': 512,
  'region': geometry
}
cloud_pct, least_result = earth.plot_image(image.resample('bicubic'), parameters, CLOUD_NAME)
from datetime import datetime as dt
date = ee.Date(image.get('system:time_start'))
time = date.getInfo()['value']/1000.
print(dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
#image = ee.Image(collection.sort('CLOUD_COVER').first()
lat = 37.658251
lon = 128.669856
CLOUD_NAME='CLOUDY_PIXEL_PERCENTAGE'
earth = EasyEarth('COPERNICUS/S2_SR')
geometry = earth.create_geo(lat,lon,10)
year = 2019
region = ee.Geometry.Polygon([[[
                128.30838423579402,
                37.480315764205976
              ],
              [
                128.30838423579402,
                37.39048423579403
              ],
              [
                128.39821576420596,
                37.39048423579403
              ],
              [
                128.39821576420596,
                37.480315764205976
              ]
            ]])
image = (ee.ImageCollection('COPERNICUS/S2_SR')
              .filterBounds(region)
              #.filterDate(f'{year}-04-14', f'{year}-05-15')
              .filterDate('2017-04-13','2017-04-14')
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .first()
              .clip(geometry))
parameters = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
  'dimensions': 512,
  'region': geometry
}
#cloud_pct, least_result = earth.plot_image(image.resample('bicubic'), parameters, CLOUD_NAME)
from datetime import datetime as dt
date = ee.Date(image.get('system:time_start'))
time = date.getInfo()['value']/1000.
print(dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
lat = 37.658251
lon = 128.669856
earth = EasyEarth('COPERNICUS/S2_SR')
geometry = earth.create_geo(lat,lon,10)
year = 2019
image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(geometry)
              .filterDate('2017-04-15','2017-05-15')
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .first()
              .clip(geometry))
'''from datetime import datetime as dt
date = ee.Date(image.get('system:time_start'))
time = date.getInfo()['value']/1000.
print(dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'))'''
classified = image.select(bands).classify(classifier)
Map.centerObject(points, 11)
Map.addLayer(classified, {'min': 0, 'max': 3, 'palette': ['#0b4a8b', '#ffc82d', '#00ffff','#bf04c2']},
          'classification')
Map

### 각 토지 이용변화 비율 추출하기

### 농업 : 0 , 도시 : 1, 수역: 2, 숲 :3

In [ ]:
#시각화 패키지 설치
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

In [ ]:
#동적 맵 생성하기.
Map = geemap.Map(center=[40,-100], zoom=4)
Map

In [ ]:
#table= ee.FeatureCollection("users/jinwoo95/pyeongchang")
#area = table.getInfo()['features'][0]['geometry']['coordinates']
#roi = ee.Geometry.Polygon(area)
#classified = image.select(bands).classify(classifier)
#least_cloudy_img= save_df.loc[0,'Classified'].clip(roi)
# Display the inputs and the results.
#classified = classified.clip(region)
#earth = EasyEarth('COPERNICUS/S2_SR')
#lat = 37.647125
#lon = 128.685141
'''region = ee.Geometry.Polygon([[[
                128.30838423579402,
                37.480315764205976
              ],
              [
                128.30838423579402,
                37.39048423579403
              ],
              [
                128.39821576420596,
                37.39048423579403
              ],
              [
                128.39821576420596,
                37.480315764205976
              ]
            ]])

#image = (ee.ImageCollection('COPERNICUS/S2_SR')
                .filterBounds(region)
                .filterDate('2019-04-01', '2019-04-30')
                .sort('CLOUDY_PIXEL_PERCENTAGE')
                .first()
                .clip(region))
'''
##위도,경도,반경 몇키로, 날짜, image_sort기준, 구름 몇 %이하로 설정할 것인지
#earth.select_AOI(lat,lon, 10, ("2018-04-01", "2018-04-30"), "CLOUDY_PIXEL_PERCENTAGE", 10)
#earth.sort_by('CLOUDY_PIXEL_PERCENTAGE')
#k = earth.get_cimg()
#classified = image.select(bands).classify(classifier)
#classified = least_cloudy_img.select(bands).classify(classifier)
Map.centerObject(points, 11)
Map.addLayer(save_df['Classified'][0], {'min': 0, 'max': 3, 'palette': ['#0b4a8b', '#ffc82d', '#00ffff','#bf04c2']},
          'classification')
Map

In [ ]:
from datetime import datetime as dt
date = ee.Date(k.get('system:time_start'))
time = date.getInfo()['value']/1000.
time = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
print(time)

In [ ]:
fallowedBinary = classified2010.eq(3).and(classified2015.eq(2));
Map.addLayer(fallowedBinary);

In [ ]:
least_result

In [ ]:
#areaImageSqM = ee.Image.clip(roi);
Map.centerObject(points, 11)
Map.addLayer(classified,{'min': 0, 'max': 3, 'palette': ['#0b4a8b', '#ffc82d', '#00ffff','#bf04c2']},
           'classification')

Map

In [ ]:
classified

In [ ]:
least_cloudy_img.getInfo()

### 클래스 별 변화 살펴보기 TEST
농업 : 0 , 도시 : 1, 수역: 2, 숲 :3

In [ ]:
a = save_df['Classified'][0]
b = save_df['Classified'][4]

In [ ]:
#바뀐 지역
a = save_df['Classified'][0]
b = save_df['Classified'][4]
fallowedBinary = (a.eq(3)_and(b.eq(1)));
areaImageSqM = (ee.Image.pixelArea()
	.clip(save_df['Geometry'][0]))
areaImageSqKm = areaImageSqM.multiply(0.000001);
fallowedArea = fallowedBinary.multiply(areaImageSqKm);

In [ ]:
save_df

In [ ]:
#바뀐 지역
a = save_df['Classified'][4]
b = save_df['Classified'][0]
roi = save_df['Geometry'][0]
fallowedBinary = a.eq(3).And(b.eq(1));
#fallowedBinary = (a.eq(3) and (b.eq(1)));
areaImageSqM = (ee.Image.pixelArea()
	.clip(roi))
areaImageSqKm = areaImageSqM.multiply(0.000001);
fallowedArea = fallowedBinary.multiply(areaImageSqKm);
reducer = ee.Reducer.sum()
geometry = roi
scale = 30
totalFallowedArea = fallowedArea.reduceRegion(reducer,geometry,scale)
'''totalFallowedArea = (fallowedArea.reduceRegion({'reducer': ee.Reducer.sum(),
	'geometry' : roi,
	'scale': 30,
  'maxPixels': 1e9}))
'''
print('Total fallowed area, sq km: ', totalFallowedArea.values().getInfo())


In [ ]:
#동적 맵 생성하기.
Map = geemap.Map(center=[40,-100], zoom=4)
Map

In [ ]:
roi = save_df['Geometry'][0]
isinstance(roi,ee.ComputedObject)

In [ ]:
print(totalFallowedArea.values().getInfo())

In [ ]:
save_df

In [ ]:
a = save_df['Classified'][0]
b = save_df['Classified'][4]
fallowedBinary = (a.eq(3).And(b.eq(0)));
Map.centerObject(points, 11)
Map.addLayer(fallowedBinary,
             {'min': 0, 'max': 3, 'palette': ['#0b4a8b', '#ffc82d']},
           'classification')

Map

In [ ]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map

In [ ]:
Map = geemap.Map(center=[40,-100], zoom=4)
agri = classified.eq(1)
Map.centerObject(points, 11)
Map.addLayer(agri,
 {'min':0, 'max':1, 'palette': ['grey', 'blue']},
 'Built-Up')
Map 

In [ ]:
a = classified.reduce(ee.Reducer.median())

In [ ]:
a.

### 클래스별 면적 계산

In [ ]:
areaImage = (ee.Image.pixelArea().addBands(
      classified))
areas = (areaImage.reduceRegion(
    reducer =  ee.Reducer.sum().group(groupField = 1,groupName = 'class'),
    geometry =  geometry,
    scale= 500,
    maxPixels =  1e10
    ));
#print(areas.getInfo())
each_class = areas.getInfo()
sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum']+each_class['groups'][3]['sum'])

agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
water = round(round(each_class['groups'][2]['sum']) / sum*100,2)
forest = round(round(each_class['groups'][3]['sum']) / sum*100,2)
print(agri, urban, water, forest)

In [ ]:
classAreas = ee.List(areas.get('groups'))

def k(item) :
  areaDict = ee.Dictionary(item)
  classNumber = ee.Number(areaDict.get('class')).format()
  area = ee.Number(areaDict.get('sum')).divide(1e6).round()
  return ee.List([classNumber, area])

classAreaLists = classAreas.map(k)
result = ee.Dictionary(classAreas.map(classAreaLists.flatten()))
print(result.values().getInfo())

### 클래스별 면적 후처리

In [ ]:
nestedList = ee.List(areas.getInfo())
#print(nestedList) 
print(nestedList.flatten().getInfo())

In [ ]:
classAreas = ee.List(areas.get('groups'))

def k(item) :
  areaDict = ee.Dictionary(item)
  classNumber = ee.Number(areaDict.get('class')).format()
  area = ee.Number(areaDict.get('sum')).divide(1e6).round()
  return ee.List([classNumber, area])

classAreaLists = classAreas.map(k)
result = ee.Dictionary(classAreas.map(classAreaLists.flatten()))
print(result.values())

In [ ]:
def calculateClassArea(feature) :
    areas = ee.Image.pixelArea().addBands(classified).reduceRegion(**{
      'reducer': ee.Reducer.sum().group(**{
      'groupField': 1,
      'groupName': 'class',
    }),
    'geometry': geometry,
    'scale': 500,
    'maxPixels': 1e10
    })
    classAreas = ee.List(areas.get('groups'))
    classAreaLists = classAreas.map(k)
    result = ee.Dictionary(classAreaLists.flatten())
    district = feature.get('ADM2_NAME')
    return ee.Feature(
      feature.geometry(),
      result.set('district', district))
    
districtAreas = ee.List(geometry).map(calculateClassArea);
print(districtAreas.values())
'''
var districtAreas = kerala.map(calculateClassArea);  

calculateClassArea = function(feature) {
    var areas = ee.Image.pixelArea().addBands(classified)
    .reduceRegion({
      reducer: ee.Reducer.sum().group({
      groupField: 1,
      groupName: 'class',
    }),
    geometry: feature.geometry(),
    scale: 500,
    maxPixels: 1e10
    })
 
    var classAreas = ee.List(areas.get('groups'))
    var classAreaLists = classAreas.map(function(item) {
      var areaDict = ee.Dictionary(item)
      var classNumber = ee.Number(
        areaDict.get('class')).format()
      var area = ee.Number(
        areaDict.get('sum')).divide(1e6).round()
      return ee.List([classNumber, area])
    })
 
    var result = ee.Dictionary(classAreaLists.flatten())
    var district = feature.get('ADM2_NAME')
    return ee.Feature(
      feature.geometry(),
      result.set('district', district))
}
 
var districtAreas = kerala.map(calculateClassArea);
'''

In [ ]:
stateArea = geometry.area()
stateAreaSqKm = ee.Number(stateArea).divide(1e6).round()
print(stateAreaSqKm)

In [ ]:
geometry.getInfo()

In [ ]:
areaImage = (ee.Image.pixelArea().addBands(
      classified))
areas = (areaImage.reduceRegion(
    reducer =  ee.Reducer.sum().group(groupField = 1,groupName = 'class'),
    geometry =  geometry,
    scale= 500,
    maxPixels =  1e10
    )); 
 

print(areas.getInfo())
classAreas = ee.List(areas.get('groups'))

def k(item) :
  areaDict = ee.Dictionary(item)
  classNumber = ee.Number(areaDict.get('class')).format()
  area = ee.Number(areaDict.get('sum')).divide(1e6).round()
  return ee.List([classNumber, area])

classAreaLists = classAreas.map(k)
result = ee.Dictionary(classAreas.map(classAreaLists.flatten()))
print(result.values().getInfo())

### 테스트

In [ ]:
#분류 전 데이터 학습 및 검증하기
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
#알펜시아
#lat = 37.658251
#lon = 128.669856
#진부역
#lat = 37.642935
#lon = 128.574451
#올림픽 게이트웨이
#lat = 37.668117
#lon = 128.705676
#자연휴양
#lat = 37.705692
#lon = 128.720683
#평창역
lat = 37.562846
lon = 128.429930
earth = EasyEarth('COPERNICUS/S2')
CLOUD_NAME = "CLOUDY_PIXEL_PERCENTAGE" # "CLOUD_COVER"
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',"B8A","B9","B10","B11","B12"]
#badns = ["B4","B8","B11"]
save_df = pd.DataFrame(columns=["image_list","lat", "lon", "image_name","cloud_pct","year","time","NDVI","NDBI","UI","Agri","Urban","Water","Forest","Classified"])
image_list = []
def cloudMask(image) :
  qa = image.select('QA60')
  allCloudBitMask = (1 << 10) + (1 << 11)
  mask = qa.bitwiseAnd(allCloudBitMask).eq(0)
  return image.updateMask(mask);
for year in ["2016","2017","2018","2019","2020"]:
  name = f"{lat}_{lon}_{year}.png" 
  if (year == "2016") :
    earth.select_AOI(lat,lon, 10, ("2016-04-16", "2016-04-28"), CLOUD_NAME, 10)
  elif (year == "2017"):
    earth.select_AOI(lat,lon, 10, ("2017-04-15", "2017-05-01"), CLOUD_NAME, 10)
  elif (year =="2018"):
    earth.select_AOI(lat,lon, 10, ("2018-04-01", "2018-04-19"), CLOUD_NAME, 10)
  elif ( year == "2020"):
    earth.select_AOI(lat,lon, 10, ("2020-04-15", "2020-05-15"), CLOUD_NAME, 10)
  else :
    earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)

  parameters = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    'dimensions': 512,
    'region': earth.AOI
  }
  #earth.data_AOI.map(cloudMask)
  geometry = earth.AOI
  earth.sort_by(CLOUD_NAME)
  least_cloudy_img = earth.get_collections_at(0)
  least_cloudy_img = least_cloudy_img.clip(earth.AOI)
  cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
  date = ee.Date(least_cloudy_img.get('system:time_start'))
  time = date.getInfo()['value']/1000.
  classified = least_cloudy_img.select(bands).classify(classifier)
  ndvi = earth.get_ndvi(least_cloudy_img)
  ndbi = earth.get_ndbi(least_cloudy_img)
  ui = earth.get_ui(least_cloudy_img)
  cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
  image_list.append(least_result)
  
  #클래스 별 면적 구하기
  areaImage = (ee.Image.pixelArea().addBands(
      classified))
  areas = (areaImage.reduceRegion(
    reducer =  ee.Reducer.sum().group(groupField = 1,groupName = 'class'),
    geometry =  geometry,
    scale= 500,
    maxPixels =  1e10
    ));
  each_class = areas.getInfo()
  if(len(each_class['groups']) == 4) :
    sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum']+each_class['groups'][3]['sum'])
    agri =  round(round(each_class['groups'][0][' sum']) / sum *100,2)
    urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
    water = round(round(each_class['groups'][2]['sum']) / sum*100,2)
    forest = round(round(each_class['groups'][3]['sum']) / sum*100,2)
  elif(len(each_class['groups'])==3):
    sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum'])
    agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
    urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
    water = 0
    forest = round(round(each_class['groups'][2]['sum']) / sum*100,2)
                

  #학습된 모델로 이미지 분류

  # 메타 정보 데이터프레임에 저장
  tmp_result = pd.Series({"image_list" : least_result,"lat": lat, "lon": lon,  "image_name": name,  "cloud_pct": cloud_pct,
                            "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'),"NDVI" : ndvi,"NDBI": ndbi,"UI": ui,"Agri":agri,"Urban" : urban,"Water" : water,"Forest" : forest,"Clssified": classified}).to_frame().T
  save_df = save_df.append(tmp_result,ignore_index=True)

save_df

In [ ]:
from matplotlib import pyplot as plt
x = [2016,2017,2018,2019,2020]
y =  save_df['Agri']
y1 = save_df['Urban']
y2 = save_df['Water']
y3 = save_df['Forest']
plt.figure(figsize = (8,8)) 
plt.plot(x,y)
plt.plot(x,y1)
plt.plot(x,y2)
plt.plot(x,y3)
plt.legend(['Agriculture', 'Urban','Water','Forest'])
# 화면에 그래프를 보여줍니다
plt.grid(10)
plt.xticks([2016 ,2017, 2018, 2019 ,2020])
plt.show()

In [ ]:
#동적 맵 생성하기.
Map = geemap.Map(center=[40,-100], zoom=4)
Map

classified = save_df['Clssified'][0]
Map.centerObject(points, 11)
Map.addLayer(classified, {'min': 0, 'max': 3, 'palette': ['#0b4a8b', '#ffc82d', '#00ffff','#bf04c2']},
          'classification')
Map

In [ ]:
save_df['image_list'][4]

In [ ]:
save_df['image_list'][2]

In [ ]:
save_df['image_list'][3]

In [ ]:
py = 

## 평창행정구역 별로 clip한 후 데이터 획득하기

In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
#table= ee.FeatureCollection("users/jinwoo95/pyeongchang")
table= ee.FeatureCollection("users/jinwoo95/hwacheon")
area = table.getInfo()['features'][0]['geometry']['coordinates']
roi = ee.Geometry.Polygon(area)
image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate("2016-04-16", "2016-04-28")
              .first()
              .clip(roi))
def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
def get_ndvi(image,roi):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  region = roi
  # Use a mean reducer.
  reducer = ee.Reducer.mean()

  # Compute the unweighted mean.
  ndvi_value = ndvi.select(['NDVI']).reduceRegion(**{ 'reducer': ee.Reducer.mean(),
  'geometry': roi,
  'maxPixels': 1e9})
  return ndvi_value.getInfo()['NDVI']
def get_ndbi(image,roi):
  ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
  region =roi
  # Use a mean reducer.
  reducer = ee.Reducer.mean()

  # Compute the unweighted mean.
  ndbi_value = ndbi.select(['NDBI']).reduceRegion(**{ 'reducer': ee.Reducer.mean(),
  'geometry': roi,
  'maxPixels': 1e9})
  return ndbi_value.getInfo()['NDBI']

def get_ui(image,roi):
    Ndvi = image.normalizedDifference(['B8', 'B4'])
    Ndbi = image.normalizedDifference(['B11', 'B8'])
    Ui = Ndbi.subtract(Ndvi).rename('Ui');
    region = roi
    reducer = ee.Reducer.mean()
    Ui_value = Ui.select(['Ui']).reduceRegion(**{ 'reducer': ee.Reducer.mean(),
  'geometry': roi,
  'maxPixels': 1e9})
    return Ui_value.getInfo()['Ui']


def get_ndvi_image(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return ndvi

def plot_image(img, paramters, cloud_name = "CLOUDY_PIXEL_PERCENTAGE"):
  
  cloud_pct = img.get("CLOUDY_PIXEL_PERCENTAGE").getInfo()
  # print(f'Cloud Cover (%): {cloud_pct}')

  url = img.getThumbUrl(parameters)
  response = requests.get(url)
  
  return cloud_pct, Image.open(BytesIO(response.content))

image_list = []
save_df = pd.DataFrame(columns=["raw_name", "image_name",'ndvi_image' "cloud_pct","year","time","NDVI","NDBI","UI"])

for year in ["2016","2017","2018","2019","2020"]:
  name = f"{year}.png" 
  if (year == "2016") :
    image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate("2016-04-16", "2016-04-28")
              .first()
              .clip(roi))
  elif (year == "2017"):
     image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate(range_year(year))
              #.filterDate("2017-04-24", "2017-05-05")
              .first()
              .clip(roi))
  elif (year =="2018"):
     image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate(range_year(year))
              #.filterDate("2018-04-01", "2018-04-19")
              .first()
              .clip(roi))
  elif ( year == "2020"):
     image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate(range_year(year))
              #.filterDate("2020-04-15", "2020-05-15")
              .first()
              .clip(roi))
  else :
     image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate(range_year(year))
              #.filterDate("2019-04-01", "2019-05-01")
              .first()
              .clip(roi))
  parameters = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    'dimensions': 512,
    'region': roi
  }
  cloud_pct, least_result = plot_image(image.resample('bicubic'), parameters,'CLOUDY_PIXEL_PERCENTAGE')
  date = ee.Date(image.get('system:time_start'))
  time = date.getInfo()['value']/1000.
  classified = image.select(bands).classify(classifier)
  ndvi = get_ndvi(image,roi)
  ndbi = get_ndbi(image,roi)
  ui = get_ui(image,roi)
  image_list.append(least_result)

  areaImage = (ee.Image.pixelArea().addBands(
      classified))
  areas = (areaImage.reduceRegion(
    reducer =  ee.Reducer.sum().group(groupField = 1,groupName = 'class'),
    geometry =  roi,
    scale= 500,
    maxPixels =  1e10
    ));
  each_class = areas.getInfo()
  if(len(each_class['groups']) == 4) :
    sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum']+each_class['groups'][3]['sum'])
    agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
    urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
    water = round(round(each_class['groups'][2]['sum']) / sum*100,2)
    forest = round(round(each_class['groups'][3]['sum']) / sum*100,2)
  elif(len(each_class['groups'])==3):
    sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum'])
    agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
    urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
    water = 0
    forest = round(round(each_class['groups'][2]['sum']) / sum*100,2)

   # 메타 정보 데이터프레임에 저장
  tmp_result = pd.Series({"raw_name" : image.getInfo()['id'], "image_name": name,  "cloud_pct": cloud_pct,
                            "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'),"NDVI" : ndvi,"NDBI" : ndbi,"UI" : ui,"Agri":agri,"Urban" : urban,"Water" : water,"Forest" : forest}).to_frame().T
  save_df = save_df.append(tmp_result,ignore_index=True)

In [ ]:
#table= ee.FeatureCollection("users/jinwoo95/hwacheon")
area = table.getInfo()['features'][0]['geometry']['coordinates']
roi = ee.Geometry.Polygon(area)

In [ ]:
image_list[2]

In [ ]:
table= ee.FeatureCollection("users/jinwoo95/real_chuncheon")
area = table.getInfo()['features'][0]['geometry']['coordinates']
roi = ee.Geometry.Polygon(area)
image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate("2020-04-01", "2020-05-15")
              .first()
              .clip(roi))
parameters = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    'dimensions': 512,
    'region': roi
  }
cloud_pct, least_result = plot_image(image.resample('bicubic'), parameters,'CLOUDY_PIXEL_PERCENTAGE')
least_result

In [ ]:
from matplotlib import pyplot as plt
x = [2016,2017,2018,2019,2020]
y =  save_df['NDVI']
y1 = save_df['NDBI']
y2 = save_df['UI']

plt.figure(figsize = (8,8)) 
plt.xlabel("Time")
plt.plot(x,y)
plt.plot(x,y1)
plt.plot(x,y2)
plt.grid(10)
plt.xticks([2016 ,2017, 2018, 2019 ,2020])
plt.legend(['NDVI', 'NDBI','UI'])
# 화면에 그래프를 보여줍니다
plt.show()

In [ ]:
image_list[0]

In [ ]:
image_list[1]

In [ ]:
image_list[2]

In [ ]:
image_list[3]

In [ ]:
image_list[4]

위성사진 획득하기

In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
table= ee.FeatureCollection("users/jinwoo95/pyeongchang")
area = table.getInfo()['features'][0]['geometry']['coordinates']
roi = ee.Geometry.Polygon(area)
image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate("2016-04-16", "2016-04-28")
              .first()
              .clip(roi))
def plot_image(img, paramters, cloud_name = "CLOUDY_PIXEL_PERCENTAGE"):
  
  cloud_pct = img.get("CLOUDY_PIXEL_PERCENTAGE").getInfo()
  # print(f'Cloud Cover (%): {cloud_pct}')

  url = img.getThumbUrl(parameters)
  response = requests.get(url)
  
  return cloud_pct, Image.open(BytesIO(response.content))

image_list = []
save_df = pd.DataFrame(columns=["raw_name", "image_name",'ndvi_image' "cloud_pct","year","time","NDVI","NDBI","UI"])
for year in ["2016","2017","2018","2019","2020"]:
  name = f"{year}.png" 
  if (year == "2016") :
    image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate("2016-04-16", "2016-04-28")
              .first())
              
  elif (year == "2017"):
     image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate("2017-04-15", "2017-05-01")
              .first())
            
  elif (year =="2018"):
     image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate("2018-04-01", "2018-04-19")
              .first())
           
  elif ( year == "2020"):
     image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate("2020-04-15", "2020-05-15")
              .first())
          
  else :
     image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(roi)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate("2019-04-01", "2019-05-01")
              .first())
             
  parameters = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    'dimensions': 512,
    'region': roi
  }
  cloud_pct, least_result = plot_image(image.resample('bicubic'), parameters,'CLOUDY_PIXEL_PERCENTAGE')
  date = ee.Date(image.get('system:time_start'))
  time = date.getInfo()['value']/1000.
  image_list.append(least_result)
   # 메타 정보 데이터프레임에 저장
  tmp_result = pd.Series({"raw_name" : image.getInfo()['id'], "image_name": name,  "cloud_pct": cloud_pct,
                            "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')}).to_frame().T
  save_df = save_df.append(tmp_result,ignore_index=True)

In [ ]:
save_df

In [ ]:
image_list[4]

In [ ]:
#분류 전 데이터 학습 및 검증하기
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
def calculate_alpha_ratio(image):
  img_array = np.array(image)
  return np.sum(img_array == 0) / img_array.size

#알펜시아
#lat = 37.658251
#lon = 128.669856
#진부역
#lat = 37.642935
#lon = 128.574451
#올림픽 게이트웨이
#lat = 37.668117
#lon = 128.705676
#자연휴양
#lat = 37.705692
#lon = 128.720683

#평창역
#lat = 37.562846
#lon = 128.429930
#춘천시
#lat = 37.883078
#lon =127.730863
#강원도홍청군
#lat = 37.697150
#lon  = 127.888811
#강원도 양구군
#lat = 38.110184
#lon =127.990013
#강원도 인제 군청
#lat = 38.069709
#lon = 128.169734
#강릉시청
#lat = 37.752154
#lon = 128.876049
#정선군청
#lat  = 37.394236
#lon =  128.656869
#인제군청
#lat = 38.069747
#lon = 128.169755
#횡성 군청
#lat = 37.491881
#lon = 127.985079
region = ["춘천","양구","인제","횡성","강릉","정선","평창"]
loc = [[37.883078,127.730863],
#강원도 양구군
[38.110184,127.990013],
#강원도 인제 군청
[38.069709,128.169734],
#횡성 군청
[37.491881,127.985079],
#강릉시청
[37.752154,128.876049],
#정선군청
[37.394236,128.656869],
#평창군청
[37.370814, 128.390359]]

def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
earth = EasyEarth('COPERNICUS/S2')
CLOUD_NAME = "CLOUDY_PIXEL_PERCENTAGE" # "CLOUD_COVER"
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',"B8A","B9","B10","B11","B12"]
#badns = ["B4","B8","B11"]

save_df = pd.DataFrame(columns=["Region","lat", "lon","cloud_pct","year","time","NDVI","NDBI","UI","Agri","Urban","Water","Forest","image"])
image_list = []
for i in range(len(loc)):
  lat , lon = loc[i]
  Region = region[i]
  for year in ["2016","2017","2018","2019","2020"]:
    name = f"{lat}_{lon}_{year}.png" 
    '''if (year == "2016") :
      earth.select_AOI(lat,lon, 10, ("2016-04-16", "2016-04-28"), CLOUD_NAME, 10)
    elif (year == "2017"):
      earth.select_AOI(lat,lon, 10, ("2017-04-15", "2017-05-01"), CLOUD_NAME, 10)
    elif (year =="2018"):
      earth.select_AOI(lat,lon, 10, ("2018-04-01", "2018-04-19"), CLOUD_NAME, 10)
    elif ( year == "2020"):
      earth.select_AOI(lat,lon, 10, ("2020-04-15", "2020-05-15"), CLOUD_NAME, 10)
    else :
      earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)
  '''
    earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)
    parameters = {
      'min': 0.0,
      'max': 3000,
      'bands': ['B4', 'B3', 'B2'],
      'maxPixels': 1e12,
      #'dimensions' : [5490,5490],
      'scale' : 3,
      'region': earth.AOI
    }
    
    #earth.data_AOI.map(cloudMask)
    geometry = earth.AOI
    earth.sort_by(CLOUD_NAME)
    i=0
    while 1:
      least_cloudy_img = earth.get_collections_at(i)
      least_cloudy_img = least_cloudy_img.clip(earth.AOI)
      cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
      alpha = calculate_alpha_ratio(least_result)
      i+=1
      if(alpha <0.1):
        break;
    date = ee.Date(least_cloudy_img.get('system:time_start'))
    time = date.getInfo()['value']/1000.
    classified = least_cloudy_img.select(bands).classify(classifier)
    ndvi = earth.get_ndvi(least_cloudy_img)
    ndbi = earth.get_ndbi(least_cloudy_img)
    ui = earth.get_ui(least_cloudy_img)
    cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
    image_list.append(least_result)
    
    #클래스 별 면적 구하기
    areaImage = (ee.Image.pixelArea().addBands(
        classified))
    areas = (areaImage.reduceRegion(
      reducer =  ee.Reducer.sum().group(groupField = 1,groupName = 'class'),
      geometry =  geometry,
      scale= 500,
      maxPixels =  1e10
      ));
    each_class = areas.getInfo()
    if(len(each_class['groups']) == 4) :
      sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum']+each_class['groups'][3]['sum'])
      agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
      urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
      water = round(round(each_class['groups'][2]['sum']) / sum*100,2)
      forest = round(round(each_class['groups'][3]['sum']) / sum*100,2)
    elif(len(each_class['groups'])==3):
      sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum'])
      agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
      urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
      water = 0
      forest = round(round(each_class['groups'][2]['sum']) / sum*100,2)
                  

    #학습된 모델로 이미지 분류

    # 메타 정보 데이터프레임에 저장
    tmp_result = pd.Series({"Region" : Region, "lat": lat, "lon": lon, "cloud_pct": cloud_pct,
                              "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'),"NDVI" : ndvi,"NDBI": ndbi,"UI": ui,"Agri":agri,"Urban" : urban,"Water" : water,"Forest" : forest,"image" :least_cloudy_img }).to_frame().T
    save_df = save_df.append(tmp_result,ignore_index=True)

save_df

In [ ]:
#분류 전 데이터 학습 및 검증하기
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
def calculate_alpha_ratio(image):
  img_array = np.array(image)
  return np.sum(img_array == 0) / img_array.size

#알펜시아
#lat = 37.658251
#lon = 128.669856
#진부역
#lat = 37.642935
#lon = 128.574451
#올림픽 게이트웨이
#lat = 37.668117
#lon = 128.705676
#자연휴양
#lat = 37.705692
#lon = 128.720683
#평창역
#lat = 37.562846
#lon = 128.429930

#춘천시
#lat = 37.883078
#lon =127.730863
#강원도홍청군
#lat = 37.697150
#lon  = 127.888811
#강원도 양구군
#lat = 38.110184
#lon =127.990013
#강원도 인제 군청
#lat = 38.069709
#lon = 128.169734
#강릉시청
#lat = 37.752154
#lon = 128.876049
#정선군청
#lat  = 37.394236
#lon =  128.656869
#인제군청
#lat = 38.069747
#lon = 128.169755
#횡성 군청
#lat = 37.491881
#lon = 127.985079
region = ["춘천","양구","인제","횡성","강릉","정선","평창"]
loc = [
[37.370814, 128.390359]]

def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
earth = EasyEarth('COPERNICUS/S2')
CLOUD_NAME = "CLOUDY_PIXEL_PERCENTAGE" # "CLOUD_COVER"
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',"B8A","B9","B10","B11","B12"]
#badns = ["B4","B8","B11"]

save_df = pd.DataFrame(columns=["Region","lat", "lon","cloud_pct","year","time","NDVI","NDBI","image"])
image_list = []
for i in range(len(loc)):
  lat , lon = loc[i]
  Region = region[i]
  for year in ["2016","2017","2018","2019","2020"]:
    name = f"{lat}_{lon}_{year}.png" 
    '''if (year == "2016") :
      earth.select_AOI(lat,lon, 10, ("2016-04-16", "2016-04-28"), CLOUD_NAME, 10)
    elif (year == "2017"):
      earth.select_AOI(lat,lon, 10, ("2017-04-15", "2017-05-01"), CLOUD_NAME, 10)
    elif (year =="2018"):
      earth.select_AOI(lat,lon, 10, ("2018-04-01", "2018-04-19"), CLOUD_NAME, 10)
    elif ( year == "2020"):
      earth.select_AOI(lat,lon, 10, ("2020-04-15", "2020-05-15"), CLOUD_NAME, 10)
    else :
      earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)
  '''
    earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)
    parameters = {
      'min': 0.0,
      'max': 3000,
      'bands': ['B4', 'B3', 'B2'],
      'maxPixels': 1e12,
      #'dimensions' : [5490,5490],
      'scale' : 3,
      'region': earth.AOI
    }
    
    #earth.data_AOI.map(cloudMask)
    geometry = earth.AOI
    earth.sort_by(CLOUD_NAME)
    i=0
    while 1:
      least_cloudy_img = earth.get_collections_at(i)
      least_cloudy_img = least_cloudy_img.clip(earth.AOI)
      cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
      alpha = calculate_alpha_ratio(least_result)
      i+=1
      if(alpha <0.1):
        break;
    date = ee.Date(least_cloudy_img.get('system:time_start'))
    time = date.getInfo()['value']/1000.
    ndvi = earth.get_ndvi(least_cloudy_img)
    ndbi = earth.get_ndbi(least_cloudy_img)
    ui = earth.get_ui(least_cloudy_img)
    cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
    image_list.append(least_result)

    #학습된 모델로 이미지 분류

    # 메타 정보 데이터프레임에 저장
    tmp_result = pd.Series({"Region" : Region, "lat": lat, "lon": lon, "cloud_pct": cloud_pct,
                              "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'),"NDVI" : ndvi,"NDBI": ndbi,"UI": ui,"image" :least_cloudy_img }).to_frame().T
    save_df = save_df.append(tmp_result,ignore_index=True)

save_df

In [ ]:
image_list[0]

# 다중회귀분석

In [ ]:
!pip install plotly_express
!pip install geopandas

import pandas as pd
import numpy as np
import geopandas as gpd
from pandas import DataFrame
import matplotlib.pyplot as plt
import plotly_express as px
import matplotlib as mpl
# 그래프 한글 환경 설정
%config InlineBackend.figure_format = 'retina'
#!apt -qq -y install fonts-nanum
import matplotlib.font_manager as fm
from geopandas import GeoDataFrame
import seaborn as sns
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
import statsmodels.formula.api as sm  
import pandas as pd # csv file 
from sklearn.linear_model import LinearRegression # 선형회귀모델 생성 
from sklearn.model_selection import train_test_split # train/test set 생성 
from sklearn.metrics import mean_squared_error # MSE : 평균제곱오차 - model 평가 
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')
mpl.font_manager._rebuild()
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
data = pd.read_excel("olympic.xlsx",index_col=True)
data

In [ ]:
#분류 전 데이터 학습 및 검증하기
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt

region = ["춘천","양구","인제","횡성","강릉","정선","평창"]
loc = [[37.883078,127.730863],
#강원도 양구군
[38.110184,127.990013],
#강원도 인제 군청
[38.069709,128.169734],
#횡성 군청
[37.491881,127.985079],
#강릉시청
[37.752154,128.876049],
#정선군청
[37.394236,128.656869],
#평창군청
[37.370814, 128.390359]]
def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
earth = EasyEarth('COPERNICUS/S2')
CLOUD_NAME = "CLOUDY_PIXEL_PERCENTAGE" # "CLOUD_COVER"
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',"B8A","B9","B10","B11","B12"]
#badns = ["B4","B8","B11"]

save_df = pd.DataFrame(columns=["Region","lat", "lon","cloud_pct","year","time","NDVI","NDBI","UI","Agri","Urban","Water","Forest","image"])
image_list = []
for i in range(len(loc)):
  lat , lon = loc[i]
  Region = region[i]
  for year in ["2016","2017","2018","2019","2020"]:
    name = f"{lat}_{lon}_{year}.png" 
    '''if (year == "2016") :
      earth.select_AOI(lat,lon, 10, ("2016-04-16", "2016-04-28"), CLOUD_NAME, 10)
    elif (year == "2017"):
      earth.select_AOI(lat,lon, 10, ("2017-04-15", "2017-05-01"), CLOUD_NAME, 10)
    elif (year =="2018"):
      earth.select_AOI(lat,lon, 10, ("2018-04-01", "2018-04-19"), CLOUD_NAME, 10)
    elif ( year == "2020"):
      earth.select_AOI(lat,lon, 10, ("2020-04-15", "2020-05-15"), CLOUD_NAME, 10)
    else :
      earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)
  '''
    earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)
    parameters = {
      'min': 0.0,
      'max': 3000,
      'bands': ['B4', 'B3', 'B2'],
      'maxPixels': 1e12,
      #'dimensions' : [5490,5490],
      'scale' : 3,
      'region': earth.AOI
    }
    
    #earth.data_AOI.map(cloudMask)
    geometry = earth.AOI
    earth.sort_by(CLOUD_NAME)
    i=0
    while 1:
      least_cloudy_img = earth.get_collections_at(i)
      least_cloudy_img = least_cloudy_img.clip(earth.AOI)
      cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
      alpha = calculate_alpha_ratio(least_result)
      i+=1
      if(alpha <0.1):
        break;
    date = ee.Date(least_cloudy_img.get('system:time_start'))
    time = date.getInfo()['value']/1000.
    classified = least_cloudy_img.select(bands).classify(classifier)
    ndvi = earth.get_ndvi(least_cloudy_img)
    ndbi = earth.get_ndbi(least_cloudy_img)
    ui = earth.get_ui(least_cloudy_img)
    cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
    image_list.append(least_result)
    
    #클래스 별 면적 구하기
    areaImage = (ee.Image.pixelArea().addBands(
        classified))
    areas = (areaImage.reduceRegion(
      reducer =  ee.Reducer.sum().group(groupField = 1,groupName = 'class'),
      geometry =  geometry,
      scale= 500,
      maxPixels =  1e10
      ));
    each_class = areas.getInfo()
    if(len(each_class['groups']) == 4) :
      sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum']+each_class['groups'][3]['sum'])
      agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
      urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
      water = round(round(each_class['groups'][2]['sum']) / sum*100,2)
      forest = round(round(each_class['groups'][3]['sum']) / sum*100,2)
    elif(len(each_class['groups'])==3):
      sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum'])
      agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
      urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
      water = 0
      forest = round(round(each_class['groups'][2]['sum']) / sum*100,2)
                  

    #학습된 모델로 이미지 분류

    # 메타 정보 데이터프레임에 저장
    tmp_result = pd.Series({"Region" : Region, "lat": lat, "lon": lon, "cloud_pct": cloud_pct,
                              "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'),"NDVI" : ndvi,"NDBI": ndbi,"UI": ui,"Agri":agri,"Urban" : urban,"Water" : water,"Forest" : forest,"image" :least_cloudy_img }).to_frame().T
    save_df = save_df.append(tmp_result,ignore_index=True)

save_df

# Final 자료 정리

## 각 특구별 분류 지수 추출

In [ ]:
#분류 전 데이터 학습 및 검증하기
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt
def range_year(year):
  return f"{year}-04-01", f"{year}-05-01"
#알펜시아
#lat = 37.658251
#lon = 128.669856
#진부역
#lat = 37.642935
#lon = 128.574451
#올림픽 게이트웨이
#lat = 37.668117
#lon = 128.705676
#자연휴양
#lat = 37.705692
#lon = 128.720683
#평창역
#lat = 37.562846
#lon = 128.429930
#위의 순서와 같이 구성
loc = [[37.658251,128.669856],[37.642935,128.574451],[37.668117,128.705676],[37.705692,128.720683],[37.562846,128.429930]]
earth = EasyEarth('COPERNICUS/S2')
CLOUD_NAME = "CLOUDY_PIXEL_PERCENTAGE" # "CLOUD_COVER"
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',"B8A","B9","B10","B11","B12"]
#badns = ["B4","B8","B11"]
save_df = pd.DataFrame(columns=["Region","lat", "lon", "image_name","cloud_pct","year","time","NDVI","NDBI","UI","Agri","Urban","Water","Forest"])
region = ["알펜시아","진부역","올림픽 게이트웨이","자연휴양","평창역"]
image_list = []
for i in range(len(loc)):
  lat , lon = loc[i]
  Region = region[i]
  for year in ["2016","2017","2018","2019","2020"]:
    name = f"{lat}_{lon}_{year}.png" 
    if (year == "2016") :
      earth.select_AOI(lat,lon, 10, ("2016-04-16", "2016-04-28"), CLOUD_NAME, 10)
    elif (year == "2017"):
      earth.select_AOI(lat,lon, 10, ("2017-04-15", "2017-05-01"), CLOUD_NAME, 10)
    elif (year =="2018"):
      earth.select_AOI(lat,lon, 10, ("2018-04-01", "2018-04-19"), CLOUD_NAME, 10)
    elif ( year == "2020"):
      earth.select_AOI(lat,lon, 10, ("2020-04-15", "2020-05-15"), CLOUD_NAME, 10)
    else :
      earth.select_AOI(lat,lon, 10, range_year(year), CLOUD_NAME, 10)

    parameters = {
      'min': 0.0,
      'max': 3000,
      'bands': ['B4', 'B3', 'B2'],
      'dimensions': 512,
      'region': earth.AOI
    }
    #earth.data_AOI.map(cloudMask)
    geometry = earth.AOI
    earth.sort_by(CLOUD_NAME)
    least_cloudy_img = earth.get_collections_at(0)
    least_cloudy_img = least_cloudy_img.clip(earth.AOI)
    cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
    date = ee.Date(least_cloudy_img.get('system:time_start'))
    time = date.getInfo()['value']/1000.
    classified = least_cloudy_img.select(bands).classify(classifier)
    ndvi = earth.get_ndvi(least_cloudy_img)
    ndbi = earth.get_ndbi(least_cloudy_img)
    ui = earth.get_ui(least_cloudy_img)
    cloud_pct, least_result = earth.plot_image(least_cloudy_img.resample('bicubic'), parameters, CLOUD_NAME)
    image_list.append(least_result)
    
    #클래스 별 면적 구하기
    areaImage = (ee.Image.pixelArea().addBands(
        classified))
    areas = (areaImage.reduceRegion(
      reducer =  ee.Reducer.sum().group(groupField = 1,groupName = 'class'),
      geometry =  geometry,
      scale= 500,
      maxPixels =  1e10
      ));
    each_class = areas.getInfo()
    if(len(each_class['groups']) == 4) :
      sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum']+each_class['groups'][3]['sum'])
      agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
      urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
      water = round(round(each_class['groups'][2]['sum']) / sum*100,2)
      forest = round(round(each_class['groups'][3]['sum']) / sum*100,2)
    elif(len(each_class['groups'])==3):
      sum = round(each_class['groups'][0]['sum'] + each_class['groups'][1]['sum'] +each_class['groups'][2]['sum'])
      agri =  round(round(each_class['groups'][0]['sum']) / sum *100,2)
      urban = round(round(each_class['groups'][1]['sum']) / sum*100,2)
      water = 0
      forest = round(round(each_class['groups'][2]['sum']) / sum*100,2)
                  

    #학습된 모델로 이미지 분류

    # 메타 정보 데이터프레임에 저장
    tmp_result = pd.Series({"Region": Region,"lat": lat, "lon": lon,  "image_name": name,  "cloud_pct": cloud_pct,
                              "year": year, "time" : dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'),"Agri":agri,"Urban" : urban,"Water" : water,"Forest" : forest}).to_frame().T
    save_df = save_df.append(tmp_result,ignore_index=True)

save_df

In [ ]:
save_df.to_csv('C:/Users/wlsdn/OneDrive/바탕 화면/qqq.csv',encoding= 'UTF-8')

In [ ]:
#알펜시아
#lat = 37.658251
#lon = 128.669856
#진부역
#lat = 37.642935
#lon = 128.574451
#올림픽 게이트웨이
#lat = 37.668117
#lon = 128.705676
#자연휴양
#lat = 37.705692
#lon = 128.720683
#평창역
#lat = 37.562846
#lon = 128.429930
loc = [[37.658251,128.669856],[37.642935,128.574451],[37.668117,128.705676],[37.705692,128.720683],[37.562846,128.429930]]
for i in loca:
  print(i)